In [1]:
import sqlite3 as sql
from sqlalchemy import *
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
conn = sql.connect('staging_db.db')
data = pd.read_sql_query("SELECT * FROM frequentation", conn)

In [3]:
data["date"] = pd.to_datetime(data["date"])
data.head()

,index,site_type,date,prevision,reel,cantine_nom,annee_scolaire,Effectif,vacances_dans,depuis_vacances,ferie_dans,depuis_ferie,chretiennes_dans,depuis_chretiennes,juives_dans,depuis_juives,ramadan_dans,depuis_ramadan,musulmanes_dans,depuis_musulmanes,chretiennes,juives,ramadan,musulmanes,greve,Quartier_detail,prix_Quartier_detail_m2_appart,prix_moyen_m2_appartement,prix_moyen_m2_maison,Longitude,Latitude,Plat
0,0,M/E,2011-01-03,210.0,182.0,AMPERE,2010-2011,280,54,0,112,0,3,0,76,0,210,0,43,0,0,0,0,0,0,Zola,3424.0,3436.0,4226.0,1.5847,47.2115,hachis parmentier pamplemousse salade verte fl...
1,1,M/E,2011-01-03,100.0,93.0,JACQUES TATI,2010-2011,185,54,0,112,0,3,0,76,0,210,0,43,0,0,0,0,0,0,Nantes Sud,3155.0,3270.0,3419.0,1.5267,47.1948,hachis parmentier pamplemousse salade verte fl...
2,2,M/E,2011-01-03,267.0,241.0,FRANCOIS DALLET/DOCTEUR TEILLAIS,2010-2011,339,54,0,112,0,3,0,76,0,210,0,43,0,0,0,0,0,0,None,NaN,NaN,NaN,NaN,NaN,hachis parmentier pamplemousse salade verte fl...
3,3,M/E,2011-01-03,197.0,184.0,HAROUYS,2010-2011,256,54,0,112,0,3,0,76,0,210,0,43,0,0,0,0,0,0,Hauts Pavés - Saint Félix,4131.0,4373.0,5454.0,1.5651,47.2183,hachis parmentier pamplemousse salade verte fl...
4,4,M/E,2011-01-03,271.0,227.0,LOUISE MICHEL,2010-2011,365,54,0,112,0,3,0,76,0,210,0,43,0,0,0,0,0,0,Ile de Nantes,3207.0,2972.0,3999.0,1.5310,47.2057,hachis parmentier pamplemousse salade verte fl...


## Construction des dataframes qui vont alimenter le DTWH

In [4]:
data.rename(columns={"Effectif": "effectif", "Quartier_detail": "quartier_detail", "prix_Quartier_detail_m2_appart":
          "prix_quartier_detail_m2_appart", "Longitude": "longitude", "Latitude": "latitude", "Plat": "plats"}, inplace=True)

In [5]:
# setting up the foreign key in fact table for dim_site
data['site_id'] = data['cantine_nom'] + '_' + data['annee_scolaire']
data['site_id'] = pd.Categorical(data['site_id']).codes + 1

# setting up the foreign key in fact table for all other dimensions
data["date"] = pd.to_datetime(data["date"])
data['jour_id'] = pd.Categorical(data['date']).codes + 1

In [6]:
# table de faits
frequentation_quotidienne_df = data[[
    'jour_id', 'site_id', 'date', 'prevision', 'reel']]

# dimension géographique
dim_site_df = data[['site_type', 'cantine_nom', 'annee_scolaire',
                    'effectif', 'quartier_detail', 'prix_quartier_detail_m2_appart',
                    'prix_moyen_m2_appartement', 'prix_moyen_m2_maison', 'longitude', 'latitude']]

# dimension menus
dim_menu_df = data[['date', 'plats']]

# dimension temporelle
dim_temporelle_df = data[['date', 'vacances_dans', 'depuis_vacances',
                          'ferie_dans', 'depuis_ferie', 'chretiennes_dans', 'depuis_chretiennes',
                          'juives_dans', 'depuis_juives', 'ramadan_dans', 'depuis_ramadan',
                          'musulmanes_dans', 'depuis_musulmanes']]

# dimension événement
dim_events_df = data[['date', 'chretiennes', 'juives',
                      'ramadan', 'musulmanes', 'greve']]

In [7]:
# drop duplicates to get the full benefit in term of efficiency
dim_site_df.drop_duplicates(inplace=True)
dim_menu_df.drop_duplicates(inplace=True)
dim_temporelle_df.drop_duplicates(inplace=True)
dim_events_df.drop_duplicates(inplace=True)

print(dim_site_df.shape)
print(dim_menu_df.shape)
print(dim_temporelle_df.shape)
print(dim_events_df.shape)

(774, 10)
(1188, 2)
(1188, 13)
(1188, 6)


/home/quentin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/quentin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/quentin/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Construction des tables SQL qui vont constituer le DTWH

In [8]:
# create a connector to the db
conn = sql.connect('frequentation_dtwh.db')

# drop table if exist
cursor = conn.cursor()
for table in [ 'Frequentation_quotidienne', 'Dim_site', 'Dim_menu', 'Dim_temporelle', 'Dim_evenement']:
    command = "DROP TABLE IF EXISTS {};".format(table)
    cursor.execute(command)
conn.commit()
cursor.close()

In [9]:
data.columns

Index(['index', 'site_type', 'date', 'prevision', 'reel', 'cantine_nom',
       'annee_scolaire', 'effectif', 'vacances_dans', 'depuis_vacances',
       'ferie_dans', 'depuis_ferie', 'chretiennes_dans', 'depuis_chretiennes',
       'juives_dans', 'depuis_juives', 'ramadan_dans', 'depuis_ramadan',
       'musulmanes_dans', 'depuis_musulmanes', 'chretiennes', 'juives',
       'ramadan', 'musulmanes', 'greve', 'quartier_detail',
       'prix_quartier_detail_m2_appart', 'prix_moyen_m2_appartement',
       'prix_moyen_m2_maison', 'longitude', 'latitude', 'plats', 'site_id',
       'jour_id'],
      dtype='object')

In [10]:
cursor = conn.cursor()

# Create Dim_temporelle
cursor.execute('''CREATE TABLE IF NOT EXISTS `Dim_temporelle` (
  `jour_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `date` DATE NOT NULL,
  `vacances_dans` INTEGER NULL,
  `depuis_vacances` INTEGER NULL,
  `ferie_dans` INTEGER NULL,
  `depuis_ferie` INTEGER NULL,
  `chretiennes_dans` INTEGER NULL,
  `depuis_chretiennes` INTEGER NULL,
  `musulmanes_dans` INTEGER NULL,
  `depuis_musulmanes` INTEGER NULL,
  `ramadan_dans` INTEGER NULL,
  `depuis_ramadan` INTEGER NULL,
  `juives_dans` INTEGER NULL,
  `depuis_juives` INTEGER NULL);
  ''')
     
# Create Dim_site
cursor.execute('''CREATE TABLE IF NOT EXISTS `Dim_site` (
  `site_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `site_type` VARCHAR(5) NULL,
  `cantine_nom` VARCHAR(50) NULL,
  `annee_scolaire` VARCHAR(30) NULL,
  `effectif` INTEGER NULL,
  `quartier_detail` VARCHAR(30) NULL,
  `prix_quartier_detail_m2_appart` INTEGER NULL,
  `prix_moyen_m2_appartement` INTEGER NULL,
  `prix_moyen_m2_maison` INTEGER NULL,
  `longitude` FLOAT NULL,
  `latitude` FLOAT NULL);
  ''')
        
# Create Dim_menu
cursor.execute('''CREATE TABLE IF NOT EXISTS `Dim_menu` (
  `jour_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `date` DATE NOT NULL,
  `plats` VARCHAR(200) NULL);
  ''')

# Create Dim_evenement
cursor.execute('''CREATE TABLE IF NOT EXISTS `Dim_evenement` (
  `jour_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `date` DATE NOT NULL,
  `chretiennes` TINYINT(1) NULL,
  `juives` TINYINT(1) NULL,
  `ramadan` TINYINT(1) NULL,
  `musulmanes` TINYINT(1) NULL,
  `greve` TINYINT(1) NULL);
  ''')


# Create fact table
cursor.execute('''CREATE TABLE IF NOT EXISTS `Frequentation_quotidienne` (
  `jour_site_id` INTEGER PRIMARY KEY AUTOINCREMENT,
  `date` DATE NOT NULL,
  `reel` INT NULL,
  `prevision` INT NULL,
  `jour_id` INT NOT NULL,
  `site_id` INT NOT NULL,
  CONSTRAINT `jour_id`
    FOREIGN KEY (`jour_id`)
    REFERENCES `Dim_temporelle` (`jour_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `site_id`
    FOREIGN KEY (`site_id`)
    REFERENCES `Dim_site` (`site_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_Frequentation_quotidienne_Dim_menu1`
    FOREIGN KEY (`jour_id`)
    REFERENCES `Dim_menu` (`jour_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT `fk_Frequentation_quotidienne_Dim_evenement1`
    FOREIGN KEY (`jour_id`)
    REFERENCES `Dim_evenement` (`jour_id`)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);
    ''')


# create index on date for faster retrieval
cursor.execute(''' CREATE INDEX date 
ON Frequentation_quotidienne(date); ''')
                 
conn.commit()


## Insertion des données dans le DTWH

In [11]:
# insertion des données dans le dtwh

dim_site_df.to_sql('Dim_site', conn,
                   if_exists='append', index=False)
frequentation_quotidienne_df.to_sql(
    'Frequentation_quotidienne', conn, if_exists='append', index=False)
dim_menu_df.to_sql('Dim_menu', conn,
                   if_exists='append', index=False)
dim_temporelle_df.to_sql(
    'Dim_temporelle', conn, if_exists='append', index=False)
dim_events_df.to_sql('Dim_evenement', conn,
                     if_exists='append', index=False)

In [12]:
cursor = conn.cursor()
for table in ['Frequentation_quotidienne', 'Dim_site', 'Dim_menu', 'Dim_temporelle', 'Dim_evenement']:
    print("------------------- {} --------------------".format(table), '\n')
    command = "SELECT * FROM {} limit 1".format(table)
    cursor.execute(command)
    for row in cursor.fetchall():
        print(row, '\n')
conn.commit()
cursor.close()

------------------- Frequentation_quotidienne -------------------- 

(1, '2011-01-03 00:00:00', 182, 210, 1, 35) 

------------------- Dim_site -------------------- 

(1, 'M/E', 'AMPERE', '2010-2011', 280, 'Zola', 3424, 3436, 4226, 1.5847, 47.2115) 

------------------- Dim_menu -------------------- 

(1, '2011-01-03 00:00:00', 'hachis parmentier pamplemousse salade verte flan caramel') 

------------------- Dim_temporelle -------------------- 

(1, '2011-01-03 00:00:00', 54, 0, 112, 0, 3, 0, 43, 0, 210, 0, 76, 0) 

------------------- Dim_evenement -------------------- 

(1, '2011-01-03 00:00:00', 0, 0, 0, 0, 0) 

